In [1]:
import requests
from bs4 import BeautifulSoup
import re
import time
import numpy as np
import pandas as pd
import pickle

In [2]:
url = 'https://www.beeradvocate.com/beer/profile/1170/'
response = requests.get(url)
page = response.text
soup = BeautifulSoup(page, "lxml")

In [65]:
def get_brewery_info(url):
    
    # Getting soup
    
    key = []
    values = []
    response = requests.get(url)
    page = response.text
    soup = BeautifulSoup(page, "lxml")
    
    # Finding feature values
    
    try:
        title_div = soup.find(class_='titleBar')
        beer_brew_names = str(title_div.text).replace('\n','')
    except:
        beer_brew_names = 'NA'
        
    try:
        score_obj = soup.find('span', {'class': 'BAscore_big'})
        score = re.split(r'>|<', str(score_obj))[4]
    except:
        score = 'NA'
        
    try:   
        beer_class_obj = soup.find_all('b')[4]
        beer_class = re.split(r'>|<', str(beer_class_obj))[2]
    except:
        beer_class = 'NA'
    
    stats_objs = soup.find_all('dd')[5:13]
    stats_list = []
    for item in stats_objs:
        item = str(item.text).strip()
        if len(item) > 0:
            clean_item = re.sub(r'#|%|,|/5','',item)
            if '.' in clean_item:
                stats_list.append(clean_item)
            else:
                stats_list.append(clean_item)
        else:
            pass
        
    if len(stats_list) < 7:
        stats_list = ('NA '*7).split()
    
    ranking, reviews, ratings, pDev, wants, gots, trade = stats_list
    
    # Lots of data floats freely within info_box
    
    info_box = soup.find('div',attrs={'id':'info_box'})
    
    try:
        b_type = re.split(r'INFO</b>\n<br/><br/>\n<b>Type:</b> |\n<br/><br/>\n',
                 str(info_box),maxsplit=4)[1]
    except:
        b_type = 'NA'
        
    try:
        phone_string = re.split(r'</a><br/><br/>',
                                str(info_box),maxsplit=4)
        phone_num_string = phone_string[1].split(' |')[0]
        phone_num = re.sub(r'\(|\)| |-','',phone_num_string)
        int(phone_num)
        has_phone = True
    except:
        has_phone = False
    
    try:   
        comm_desc_str = str(info_box).split('Notes:</b>\n<br/>\n')[1]
        comm_desc = comm_desc_str.split('<br')[0]
    except:
        comm_desc = 'NA'        
    
    key = beer_brew_names 
    
    values = [score, beer_class, reviews, ratings, pDev, 
              wants, gots, trade, 
              comm_desc]
    
    return key, values

In [66]:
url = 'https://www.beeradvocate.com/beer/profile/1170/'
get_brewery_info(url)

('Upland Brewing Company',
 ['3.91',
  'Very Good',
  '2581',
  '9967',
  '4.03',
  '58',
  '138',
  '10.17',
  'No notes at this time.'])

In [20]:
info_box = soup.find('div',attrs={'id':'info_box'})

In [46]:
# str(abv_div).split('Availability:</b> ')[1]
phone_string = re.split(r'</a><br/><br/>',
         str(info_box),maxsplit=4)
phone_num_string = phone_string[1].split(' |')[0]
phone_num = re.sub(r'\(|\)| |-','',phone_num_string)
int(phone_num)

8123362337

In [67]:
df = pd.read_pickle('116_4700')

In [68]:
df

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16
Häffner Hopfenstopfer Comet IPA | Hotel-Gasthof Häffner Bräu,3.8,Very Good,23092,7,16,11.58,1,2,0,Hotel-Gasthof Häffner Bräu,Germany,haeffner-braeu.de,American IPA,6.80,Rotating,No notes at this time.,02-13-2014
Brewhouse Rarities: Guava Juice IPA | Flying Dog Brewery,3.64,Good,-,5,16,7.14,0,2,0,Flying Dog Brewery,Maryland,United States,flyingdogbrewery.com,6.80,Limited (brewed once),No notes at this time.,02-25-2018
Sea Dragon | Asbury Park Brewery,3.73,Good,26601,4,16,12.06,0,4,0,Asbury Park Brewery,New Jersey,United States,asburyparkbrewery.com,7.00,Fall,No notes at this time.,10-22-2017
The Wobblies | Calicraft Brewing Co.,3.56,Good,34051,4,16,20.51,0,1,0,Calicraft Brewing Co.,California,United States,calicraft.com,7.00,Year-round,"Calicraft's first wet hop ale, 2012.",09-10-2012
Rhinestone Life | Great Raft Brewing,4.01,Exceptional,-,3,16,6.98,0,4,0,Great Raft Brewing,Louisiana,United States,greatraftbrewing.com,7.00,Rotating,No notes at this time.,06-25-2018
Kane / Sand City - Blueberry Island | Kane Brewing Company,4.12,Exceptional,-,3,16,7.04,0,1,0,Kane Brewing Company,New Jersey,United States,kanebrewing.com,6.30,Limited (brewed once),Blueberry Island is a collaboration with our g...,07-08-2018
Slam Dank | Twin Sails Brewing,4.16,Exceptional,7772,2,16,5.05,0,1,0,Twin Sails Brewing,British Columbia,Canada,twinsailsbrewing.com,7.50,Rotating,No notes at this time.,01-26-2017
Hopnoxxxious IPA | Walldorff Brewpub and Bistro,3.79,Very Good,23600,2,16,7.92,0,0,0,Walldorff Brewpub and Bistro,Michigan,United States,walldorffbrewpub.com,7.50,Rotating,No notes at this time.,09-06-2010
TKO IPA | Boxing Cat Brewery,3.97,Very Good,14570,4,16,10.08,1,3,0,Boxing Cat Brewery,China,boxingcatbrewery.com,American IPA,6.00,Year-round,No notes at this time.,02-22-2013
Yardbird Crafted Ale | Greene King / Morland Brewery,3.18,Okay,41944,2,16,17.92,0,3,0,Greene King / Morland Brewery,England,United Kingdom,greeneking.co.uk,4.00,Year-round,No notes at this time.,06-25-2013
